In [1]:
#This version (v.2.3) includes initialization of pressure + updated report format
#This version (v.2.4) includes timestep design

%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import xlrd

#Gas properties generator
p_sc = 14.7
T_sc = 520
Mw_air = 28.97
R_const = 10.732
Mw_gas = 22.836
SGg = Mw_gas/Mw_air
T_res = 190 + 460

def zg(P,T,SGg):
    
    P_pr = P/P_critical(SGg)
    T_pr = T/T_critical(SGg)
    
    #P_pr = P/738.44
    #T_pr = T/418.38
    
    A1 = 0.3265         
    A2 = -1.07          
    A3 = -0.5339        
    A4 = 0.01569        
    A5 = -0.05165        
    A6 = 0.5475
    A7 = -0.7361       
    A8 = 0.1844         
    A9 = 0.1056         
    A10 = 0.6134        
    A11 = 0.721
        
    C1 = A1 + A2 / T_pr + A3 / (T_pr * T_pr * T_pr) + A4 / (T_pr * T_pr * T_pr * T_pr) + A5 / (T_pr * T_pr * T_pr * T_pr * T_pr)
    C2 = A6 + A7 / T_pr + A8 / (T_pr * T_pr)
    C3 = A9 * (A7 / T_pr + A8 / (T_pr * T_pr))

    Ze = 1
    for i in range(0,100):
        rho_pr = 0.27 * (P_pr / (Ze * T_pr))
        C4 = A11 * rho_pr * rho_pr
        Zc = 1 + C1 * rho_pr + C2 * rho_pr * rho_pr - C3 * (rho_pr * rho_pr * rho_pr * rho_pr * rho_pr) + A10 * (1 + C4) * (rho_pr * rho_pr / (T_pr * T_pr * T_pr)) * np.exp(-C4)
        if (np.abs(Ze - Zc) < 0.001):
            break
        else:
            fz = Ze - Zc
            dfdZ = 1 + (rho_pr * rho_pr / Ze) * (C1 / rho_pr + 2 * C2 - 5 * C3 * (rho_pr * rho_pr * rho_pr) + 2 * A10 * np.exp(-C4) * (1 + C4 - C4 * C4) / (T_pr * T_pr * T_pr))
            Ze = Ze - fz / dfdZ
    
    return Zc


def T_critical(SGg):  #Sutton
    return 169.2 + 349.5 * SGg - 74 * SGg * SGg 
def P_critical(SGg): #Sutton
    return 756.8 - 131 * SGg - 3.6 * SGg**2
def T_reduced(T,SGg): #T in Rankine
    return T/T_critical(SGg)
def P_reduced(P,SGg): #T in Rankine
    return P/P_critical(SGg)

def cal_rhog(P,T,SGg):
    Mwg = SGg * Mw_air
    z = zg(P,T,SGg)
    return P*Mwg/z/R_const/T

def cal_viscg(P,T,SGg):
    z = zg(P,T,SGg)
    gasdensPT = cal_rhog(P,T,SGg)
    gasdensPT = gasdensPT * 1000 / (2.20462262184878 * ((0.3048 * 100) ** 3)) 
    Mwg = SGg * Mw_air
    XK = (9.379 + 0.01607 * Mwg) * ((T ** 1.5) / (209.2 + 19.26 * Mwg + T))
    X = 3.448 + 986.4 / T + 0.01009 * Mwg
    Y = 2.447 - 0.2224 * X
    return (XK / 10000) * np.exp(X * (gasdensPT ** Y))

def cal_cg(P,T,SGg):
    dP = 1
    z_left2 = zg(P-2*dP,T,SGg)
    z_left1 = zg(P-dP,T,SGg)
    z_cen = zg(P,T,SGg)
    z_right1 = zg(P+dP,T,SGg)
    z_right2 = zg(P+2*dP,T,SGg)
    dzdP = (z_right1-z_left1)/2/dP
    #dzdP = (-z_left2+8*z_left1-8*z_right1+z_right2)/12/dP
    return 1/P-1/z_cen*dzdP
    
def cal_Bg(P,T,SGg):
    Bg = p_sc/T_sc*zg(P,T,SGg)*T/P
    return Bg/5.6146

print(zg(5000,T_res,SGg))
print(cal_Bg(5000,T_res,SGg))
print(cal_viscg(5000,T_res,SGg))
print(cal_rhog(5000,T_res,SGg))
print(cal_rhog(14.7,520,SGg))
print(cal_cg(5000,T_res,SGg))

0.987144768022054
0.0006461292028783971
0.031758665397174306
16.581169923885977
0.06036222967048886
0.00011341287742464846


In [2]:
AA = np.zeros((3,3,3))
AA[1,2,0] = 4
print(AA)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [4. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [3]:
def pressure_initializer(Z,p_datum,Z_datum):
    p_mid = p_datum
    dp_dZ_mid =  cal_rhog(p_mid,T_res,SGg)/144
    p_cal = p_datum + dp_dZ_mid*(Z-Z_datum)
    p_mid = (p_datum+p_cal)/2

    while True:
        dp_dZ_mid =  cal_rhog(p_mid,T_res,SGg)/144
        p_cal_new = p_datum + dp_dZ_mid*(Z-Z_datum)
        if abs(p_cal_new-p_cal) < 0.0001:
            break
        p_cal = p_cal_new
        p_mid = (p_datum+p_cal_new)/2
    return p_cal_new

print(pressure_initializer(9270,7750,9300))

7745.7738022279755


In [4]:

def cal_phi(p,p_ref,phi_ref,cr):
    return phi_ref*(1+cr*(p-p_ref))

def id_tran(iy,ix):
    return ix + Nx*iy

def Jw_cal(dx,dy,kx,ky,h,viscg,Bg,rw,skin):
    req = 0.28*np.sqrt(np.sqrt(ky/kx)*dx**2+np.sqrt(kx/ky)*dy**2)/((ky/kx)**0.25+(kx/ky)**0.25)
    return 2*np.pi*cbeta*np.sqrt(kx*ky)*h/viscg/Bg/(np.log(req/rw)+skin)

def p_res_average_cal(p):
    # MBE check
    p_res_num = 0
    p_res_deno = 0
    for ix in range (0,Nx):
        for iy in range(0,Ny):
            if (null[iz,iy,ix] != 0):
                p_res_num += dx[iz,iy,ix]*dy[iz,iy,ix]*h[iz,iy,ix]*cal_phi(p[iz,iy,ix],p_ref,phi_ref[iz,iy,ix],cr)*p[iz,iy,ix]
                p_res_deno += dx[iz,iy,ix]*dy[iz,iy,ix]*h[iz,iy,ix]*cal_phi(p[iz,iy,ix],p_ref,phi_ref[iz,iy,ix],cr)
    return p_res_num/p_res_deno

Nx = 12
Ny = 9
Nz = 1
t = 5
dt = 0.5 #days

# reservoir and PVT properties
pi = 7750
p_ref = 14.7
cr = 0 #5e-8
calpha = 5.6146
cbeta = 1.127e-3
Rconst = 10.732
Z_datum = 9300

dx = np.zeros((Nz,Ny,Nx))
dy = np.zeros((Nz,Ny,Nx))
kx = np.zeros((Nz,Ny,Nx))
ky = np.zeros((Nz,Ny,Nx))
Ax = np.zeros((Nz,Ny,Nx))
Ay = np.zeros((Nz,Ny,Nx))
phi = np.zeros((Nz,Ny,Nx))
phi_ref = np.zeros((Nz,Ny,Nx))
h = np.zeros((Nz,Ny,Nx))
top = np.zeros((Nz,Ny,Nx))
bottom = np.zeros((Nz,Ny,Nx))
middle = np.zeros((Nz,Ny,Nx))
null = np.zeros((Nz,Ny,Nx))
null_im = []

p = np.zeros((Nz,Ny,Nx))
p_new = np.zeros((Nz,Ny,Nx))
p_inner = np.zeros((Nz,Ny,Nx))
qgsc = np.zeros((Nz,Ny,Nx))
#pwf = []
pwf = np.zeros((Nz,Ny,Nx))
Jw = np.zeros((Nz,Ny,Nx))

#initialization
res_Excel = xlrd.open_workbook('CH8_reservoir prop.xlsx')

for ix in range (0,Nx):
    for iy in range(0,Ny):
        
        #temporary 
        iz = 0
        
        
        dx[iz,iy,ix] = res_Excel.sheet_by_name('dx').cell(iy,ix).value
        dy[iz,iy,ix] = res_Excel.sheet_by_name('dy').cell(iy,ix).value
        kx[iz,iy,ix] = res_Excel.sheet_by_name('kx').cell(iy,ix).value
        ky[iz,iy,ix] = res_Excel.sheet_by_name('ky').cell(iy,ix).value
        phi[iz,iy,ix] = res_Excel.sheet_by_name('phi').cell(iy,ix).value
        phi_ref[iz,iy,ix] = res_Excel.sheet_by_name('phi').cell(iy,ix).value
        top[iz,iy,ix] = res_Excel.sheet_by_name('T2').cell(iy,ix).value
        bottom[iz,iy,ix] = res_Excel.sheet_by_name('T3').cell(iy,ix).value
        h[iz,iy,ix] = bottom[iz,iy,ix]-top[iz,iy,ix]
        middle[iz,iy,ix] = (top[iz,iy,ix] + bottom[iz,iy,ix])/2
        null[iz,iy,ix] = res_Excel.sheet_by_name('null').cell(iy,ix).value
        Ax[iz,iy,ix] = dy[iz,iy,ix]*h[iz,iy,ix]
        Ay[iz,iy,ix] = dx[iz,iy,ix]*h[iz,iy,ix]
        if null[iz,iy,ix] == 1:
            #pressure distribution (initialization)
            p[iz,iy,ix] = pressure_initializer(middle[iz,iy,ix],pi,Z_datum)
            

# Well info
#W2
qgsc[0,2,8] = -3e+6
#W3
pwf[0,3,3] = 6000
rw_3_3 = 0.25
rw_2_8 = 0.25


#reporting-general settings
#N_step = 68
#dt_report = [0] + [0.0625]*4 + [0.125]*2 + [0.25]*4 + [0.5]*17 + [1]*40

#N_step = 18
#dt_report = [0,0.0625,0.0625,0.0625,0.0625,0.125,0.125,0.25,0.25,0.25,0.25,0.5,0.5,0.5,0.5,0.5,0.5,0.5]

N_step = 16
dt_report = [0,0.125,0.125,0.125,0.125,0.25,0.25,0.25,0.25,0.5,0.5,0.5,0.5,0.5,0.5,0.5]

#N_step = 14
#dt_report = [0,0.25,0.25,0.25,0.25,0.25,0.25,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5]

#N_step = 11
#dt_report = [0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5]
t_report = np.zeros(N_step)
p_res_average = np.zeros(N_step)
MB_error = np.zeros(N_step)


#reporting-production settings
p_28 = np.zeros(N_step)
Jw_28 = np.zeros(N_step)
pwf_28 = np.zeros(N_step)

p_33 = np.zeros(N_step)
Jw_33 = np.zeros(N_step)
q_33 = np.zeros(N_step)


p_28[0] = p[0,2,8]

#plot settings
plt_max = 4000
plt_min = 3970

        
t_cum = 0
viscg = np.zeros((Nz,Ny,Nx))
Bg = np.zeros((Nz,Ny,Nx))
rhog = np.zeros((Nz,Ny,Nx))
cg = np.zeros((Nz,Ny,Nx))
tran_W = np.zeros((Nz,Ny,Nx))
tran_E = np.zeros((Nz,Ny,Nx))
tran_S = np.zeros((Nz,Ny,Nx))
tran_N = np.zeros((Nz,Ny,Nx))
tran_C = np.zeros((Nz,Ny,Nx))
tau_term = np.zeros((Nz,Ny,Nx))
Q_term = np.zeros((Nz,Ny,Nx))

tran_Wg = np.zeros((Nz,Ny,Nx))
tran_Eg = np.zeros((Nz,Ny,Nx))
tran_Sg = np.zeros((Nz,Ny,Nx))
tran_Ng = np.zeros((Nz,Ny,Nx))
tran_Cg = np.zeros((Nz,Ny,Nx))
Qg_term = np.zeros((Nz,Ny,Nx))

tran_W_const = np.zeros((Nz,Ny,Nx))
tran_E_const = np.zeros((Nz,Ny,Nx))
tran_S_const = np.zeros((Nz,Ny,Nx))
tran_N_const = np.zeros((Nz,Ny,Nx))

A_matrix = np.zeros((Nx*Ny,Nx*Ny))
B_matrix = np.zeros((Nx*Ny,1))
soln_matrix = np.zeros((Nx*Ny,1))

A_matrix_R = np.zeros((Nx*Ny,Nx*Ny))
B_matrix_R = np.zeros((Nx*Ny,1))
soln_matrix_R = np.zeros((Nx*Ny,1))
residual_dp = np.zeros((Nz,Ny,Nx))


#########################
# loop calculation


#Residual function (R)

def R(iy,ix,diff_dir):
    
    #temporary
    iz = 0
    
    
    
    R_e = 0.00001
    if diff_dir == 'C':
        p_C = p_inner[iz,iy,ix] + R_e
    else:
        p_C = p_inner[iz,iy,ix] 
        
    viscg_C = cal_viscg(p_C,T_res,SGg)
    Bg_C = cal_Bg(p_C,T_res,SGg)
    rhog_C = cal_rhog(p_C,T_res,SGg)
    cg_C = cal_cg(p_C,T_res,SGg)
    
    if (ix == 0) or (null[iz,iy,ix]==0):
        p_W = 0
        viscg_W = 0
        Bg_W = 0
        rhog_W = 0
        tran_W_R = 0
        tran_Wg_R = 0
    else:
        if diff_dir == 'W':
            p_W = p_inner[iz,iy,ix-1] + R_e
        else:
            p_W = p_inner[iz,iy,ix-1]
            
        viscg_W = cal_viscg(p_W,T_res,SGg)
        Bg_W = cal_Bg(p_W,T_res,SGg)
        rhog_W = cal_rhog(p_W,T_res,SGg)
        tran_W_R_const = cbeta*2/(dx[iz,iy,ix-1]/Ax[iz,iy,ix-1]/kx[iz,iy,ix-1]+dx[iz,iy,ix]/Ax[iz,iy,ix]/kx[iz,iy,ix])
        tran_W_R = tran_W_R_const/(viscg_W+viscg_C)*2/(Bg_W+Bg_C)*2
        tran_Wg_R = tran_W_R_const*(rhog_W+rhog_C)/2/144
        
    if (ix == Nx-1) or (null[iz,iy,ix]==0):
        p_E = 0
        viscg_E = 0
        Bg_E = 0
        rhog_E = 0
        tran_E_R = 0
        tran_Eg_R = 0
    else:
        if diff_dir == 'E':
            p_E = p_inner[iz,iy,ix+1] + R_e
        else:
            p_E = p_inner[iz,iy,ix+1] 

        viscg_E = cal_viscg(p_E,T_res,SGg)
        Bg_E = cal_Bg(p_E,T_res,SGg)
        rhog_E = cal_rhog(p_E,T_res,SGg)
        tran_E_R_const = cbeta*2/(dx[iz,iy,ix+1]/Ax[iz,iy,ix+1]/kx[iz,iy,ix+1]+dx[iz,iy,ix]/Ax[iz,iy,ix]/kx[iz,iy,ix])
        tran_E_R = tran_E_R_const/(viscg_E+viscg_C)*2/(Bg_E+Bg_C)*2
        tran_Eg_R = tran_E_R_const*(rhog_E+rhog_C)/2/144
    
    if (iy == 0) or (null[iz,iy,ix]==0):
        p_N = 0
        viscg_N = 0
        Bg_N = 0
        rhog_N = 0
        tran_N_R = 0
        tran_Ng_R = 0
    else:
        if diff_dir == 'N':
            p_N = p_inner[iz,iy-1,ix] + R_e
        else:
            p_N = p_inner[iz,iy-1,ix]
            
        viscg_N = cal_viscg(p_N,T_res,SGg)
        Bg_N = cal_Bg(p_N,T_res,SGg)
        rhog_N = cal_rhog(p_N,T_res,SGg)
        tran_N_R_const = cbeta*2/(dx[iz,iy-1,ix]/Ax[iz,iy-1,ix]/kx[iz,iy-1,ix]+dx[iz,iy,ix]/Ax[iz,iy,ix]/kx[iz,iy,ix])
        tran_N_R = tran_N_R_const/(viscg_N+viscg_C)*2/(Bg_N+Bg_C)*2
        tran_Ng_R = tran_N_R_const*(rhog_N+rhog_C)/2/144
    
    if (iy == Ny-1) or (null[iz,iy,ix]==0):
        p_S = 0
        viscg_S = 0
        Bg_S = 0
        rhog_S = 0
        tran_S_R = 0
        tran_Sg_R = 0
    else:
        if diff_dir == 'S':
            p_S = p_inner[iz,iy+1,ix] + R_e
        else:
            p_S = p_inner[iz,iy+1,ix]
            
        viscg_S = cal_viscg(p_S,T_res,SGg)
        Bg_S = cal_Bg(p_S,T_res,SGg)
        rhog_S = cal_rhog(p_S,T_res,SGg)
        tran_S_R_const = cbeta*2/(dx[iz,iy+1,ix]/Ax[iz,iy+1,ix]/kx[iz,iy+1,ix]+dx[iz,iy,ix]/Ax[iz,iy,ix]/kx[iz,iy,ix])
        tran_S_R = tran_S_R_const/(viscg_S+viscg_C)*2/(Bg_S+Bg_C)*2
        tran_Sg_R = tran_S_R_const*(rhog_S+rhog_C)/2/144

    
    Jw_C = 0
    #only for grid(3,3)
    if iy == 3 and ix ==3:
        Jw_C = Jw_cal(dx[0,3,3],dy[0,3,3],kx[0,3,3],ky[0,3,3],h[0,3,3],viscg_C,Bg_C,rw_3_3,0)
    
    
    tau_term_R = dx[iz,iy,ix]*dy[iz,iy,ix]*h[iz,iy,ix]*phi[iz,iy,ix]*cg_C/calpha/Bg_C
    tran_C_R = -(tran_W_R + tran_E_R + tran_S_R + tran_N_R + tau_term_R /dt + Jw_C)
    tran_Cg_R = -(tran_Wg_R + tran_Eg_R + tran_Sg_R + tran_Ng_R)
    
    Qg_term_R = 0
    if ix != 0:
        Qg_term_R += middle[iz,iy,ix-1]*tran_Wg_R
    if ix != Nx-1:
        Qg_term_R += middle[iz,iy,ix+1]*tran_Eg_R
    if iy != 0:
        Qg_term_R += middle[iz,iy-1,ix]*tran_Ng_R
    if iy != Ny-1:  
        Qg_term_R += middle[iz,iy+1,ix]*tran_Sg_R
                
    Qg_term_R += middle[iz,iy,ix]*tran_Cg_R
    Q_term_R = -(tau_term_R/dt*p[iz,iy,ix] + qgsc[iz,iy,ix] + pwf[iz,iy,ix] * Jw_C - Qg_term_R)

    sum_R = Q_term_R - (p_W*tran_W_R + p_E*tran_E_R + p_N*tran_N_R + p_S*tran_S_R + p_C*tran_C_R)
    
    if (null[iz,iy,ix]==0):
        sum_R = 0
    return sum_R

#Residual-derivative (dR/dP)
def dR_dp(iy,ix,diff_dir):
    
    R_e = 0.00001
    
    if diff_dir == 'C':
        dR_dp = (R(iy,ix,'C')-R(iy,ix,'0'))/R_e
    if diff_dir == 'W':
         dR_dp = (R(iy,ix,'W')-R(iy,ix,'0'))/R_e
    if diff_dir == 'E':
         dR_dp = (R(iy,ix,'E')-R(iy,ix,'0'))/R_e
    if diff_dir == 'N':
         dR_dp = (R(iy,ix,'N')-R(iy,ix,'0'))/R_e
    if diff_dir == 'S':
         dR_dp = (R(iy,ix,'S')-R(iy,ix,'0'))/R_e  
    return dR_dp

 ##########################  
#SIMULATOR-MATRIX SOLVER

#timestep zero

report_counter = 0

viscg[iz,iy,ix] = cal_viscg(p[iz,iy,ix],T_res,SGg)
rhog[iz,iy,ix] = cal_rhog(p[iz,iy,ix],T_res,SGg)
cg[iz,iy,ix] = cal_cg(p[iz,iy,ix],T_res,SGg)
Bg[iz,iy,ix] = cal_Bg(p[iz,iy,ix],T_res,SGg)
phi[iz,iy,ix] = cal_phi(p[iz,iy,ix],p_ref,phi_ref[iz,iy,ix],cr)
Jw[0,3,3] = Jw_cal(dx[0,3,3],dy[0,3,3],kx[0,3,3],ky[0,3,3],h[0,3,3],viscg[0,3,3],Bg[0,3,3],rw_3_3,0)

p_res_average[report_counter] = p_res_average_cal(p)
t_report[report_counter] = 0

p_28[report_counter] = p[0,2,8]
Jw_28[report_counter] = Jw_cal(dx[0,2,8],dy[0,2,8],kx[0,2,8],ky[0,2,8],h[0,2,8],viscg[0,2,8],Bg[0,2,8],rw_2_8,0)
pwf_28[report_counter] = p[0,2,8]
    
p_33[report_counter] = p[0,3,3]
q_33[report_counter] = 0
Jw_33[report_counter] = Jw[0,3,3]

t_cum = 0
report_counter = 1

while report_counter <= N_step-1:

    #temporary
    iz = 0
    
    
    dt = dt_report[report_counter]  #assign time step as specified in the array
    t_cum += dt
    t_report[report_counter] = t_cum
    
    print('--')
    print(t_cum)
    print('--')
    print(t_report[report_counter])
    print('--')
    print(dt)
    print('--')
    
    for inner_counter in range(0,4):

        for ix in range (0,Nx):
            for iy in range(0,Ny):
                
                if inner_counter == 0:
                    viscg[iz,iy,ix] = cal_viscg(p[iz,iy,ix],T_res,SGg)
                    rhog[iz,iy,ix] = cal_rhog(p[iz,iy,ix],T_res,SGg)
                    cg[iz,iy,ix] = cal_cg(p[iz,iy,ix],T_res,SGg)
                    Bg[iz,iy,ix] = cal_Bg(p[iz,iy,ix],T_res,SGg)
                    phi[iz,iy,ix] = cal_phi(p[iz,iy,ix],p_ref,phi_ref[iz,iy,ix],cr)
                    Jw[0,3,3] = Jw_cal(dx[0,3,3],dy[0,3,3],kx[0,3,3],ky[0,3,3],h[0,3,3],viscg[0,3,3],Bg[0,3,3],rw_3_3,0)
                    p_inner = p
                else:
                    viscg[iz,iy,ix] = cal_viscg(p_inner[iz,iy,ix],T_res,SGg)
                    rhog[iz,iy,ix] = cal_rhog(p_inner[iz,iy,ix],T_res,SGg)
                    cg[iz,iy,ix] = cal_cg(p_inner[iz,iy,ix],T_res,SGg)
                    Bg[iz,iy,ix] = cal_Bg(p_inner[iz,iy,ix],T_res,SGg)
                    phi[iz,iy,ix] = cal_phi(p_inner[iz,iy,ix],p_ref,phi_ref[iz,iy,ix],cr)
                    Jw[0,3,3] = Jw_cal(dx[0,3,3],dy[0,3,3],kx[0,3,3],ky[0,3,3],h[0,3,3],viscg[0,3,3],Bg[0,3,3],rw_3_3,0)
        print(p_inner)

        
        #Jacobian generator
        for ix in range (0,Nx):
            for iy in range(0,Ny):

                im = id_tran(iy,ix)
                if null[iz,iy,ix] == 0:
                    null_im.append(im)
                A_matrix_R[im,im] = dR_dp(iy,ix,'C')

                if (im-1)>= 0:
                    A_matrix_R[im,im-1] = dR_dp(iy,ix,'W')

                if (im+1)<= (Nx*Ny-1):
                    A_matrix_R[im,im+1] = dR_dp(iy,ix,'E')

                if (im-Nx)>= 0:
                    A_matrix_R[im,im-Nx] = dR_dp(iy,ix,'N')

                if (im+Nx)<= (Nx*Ny-1):
                    A_matrix_R[im,im+Nx] = dR_dp(iy,ix,'S')

                B_matrix_R[im,0] = R(iy,ix,'0')
#                 if abs(B_matrix_R[im,0]) > 0.01:
#                     print(B_matrix_R[im,0])
#                     print(iy)
#                     print(ix)
#         print(B_matrix_R)    

        null_im.sort()
        #print(null_im)
        A_matrix_1_R = np.delete(A_matrix_R,null_im,0)
        A_matrix_2_R = np.delete(A_matrix_1_R,null_im,1)
        B_matrix_1_R = np.delete(B_matrix_R,null_im,0)  
        
        soln_matrix_R = np.dot(np.linalg.inv(A_matrix_2_R),B_matrix_1_R)

        rev_counter = 0
        for iy in range (0,Ny):
            for ix in range(0,Nx):

                if null[iz,iy,ix] == 0:
                    residual_dp[iz,iy,ix] = 0
                else:
                    residual_dp[iz,iy,ix] = soln_matrix_R[rev_counter,0]
                    rev_counter += 1
   
        p_inner =  p_inner - residual_dp
        
        # MBE check
        IMB_num = 0
        IMB_deno = 0
        for ix in range (0,Nx):
            for iy in range(0,Ny):
                if (null[iz,iy,ix] != 0):
                    IMB_num += dx[iz,iy,ix]*dy[iz,iy,ix]*h[iz,iy,ix]/calpha*(cal_phi(p_inner[iz,iy,ix],p_ref,phi_ref[iz,iy,ix],cr)/cal_Bg(p_inner[iz,iy,ix],T_res,SGg)-cal_phi(p[iz,iy,ix],p_ref,phi_ref[iz,iy,ix],cr)/cal_Bg(p[iz,iy,ix],T_res,SGg))
                if (qgsc[iz,iy,ix] != 0):
                    IMB_deno += dt*qgsc[iz,iy,ix]

                if (Jw[iz,iy,ix] != 0 ):
                    IMB_deno += -dt*Jw[iz,iy,ix]*(p_inner[iz,iy,ix]-pwf[iz,iy,ix])
                

        IMB = IMB_num/IMB_deno

        #print(p_new)
        print(IMB)
        #print(p_inner)
        print('')    
    
    p_new = p_inner #assign the last inner iteration to p_new
    p_28[report_counter] = p_new[0,2,8]
    Jw_28[report_counter] = Jw_cal(dx[0,2,8],dy[0,2,8],kx[0,2,8],ky[0,2,8],h[0,2,8],viscg[0,2,8],Bg[0,2,8],rw_2_8,0)
    pwf_28[report_counter] = qgsc[0,2,8]/Jw_28[report_counter] + p_28[report_counter]
    
    p_33[report_counter] = p_new[0,3,3]
    q_33[report_counter] = Jw[0,3,3]*(p_new[0,3,3]-pwf[0,3,3])
    Jw_33[report_counter] = Jw[0,3,3]
    p_res_average[report_counter] = p_res_average_cal(p_new)
    MB_error[report_counter] = IMB
    
    report_counter += 1
    
    p[:] = p_new[:]

--
0.125
--
0.125
--
0.125
--


C:\Users\Kittipong Limchuchua\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\Kittipong Limchuchua\Anaconda3\lib\site-packages\ipykernel_launcher.py:93: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\Kittipong Limchuchua\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':
C:\Users\Kittipong Limchuchua\Anaconda3\lib\site-packages\ipykernel_launcher.py:276: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Kittipong Limchuchua\Anaconda3\lib\site-packages\ipykernel_launcher.py:245: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\Kittipong Limchuchua\Anaconda3\lib\site-packages\ipykernel_launcher.py:265: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\Kittipong Limchuchua\Anaconda3\lib\site-packages\ipykernel_launcher.py:205: RuntimeWarning:

[[[   0.         7752.11342029 7752.11342029 7752.11342029    0.
      0.            0.            0.            0.            0.
      0.            0.        ]
  [7752.11342029 7752.11342029 7752.11342029 7752.11342029 7752.11342029
      0.            0.            0.         7752.11342029 7752.11342029
      0.            0.        ]
  [7752.11342029 7752.11342029 7752.11342029 7752.11342029 7752.11342029
   7752.11342029 7752.11342029 7752.11342029 7752.11342029 7752.11342029
   7752.11342029 7752.11342029]
  [7752.11342029 7752.11342029 7752.11342029 7752.11342029 7752.11342029
   7752.11342029 7752.11342029 7752.11342029 7752.11342029 7752.11342029
   7752.11342029 7752.11342029]
  [7752.11342029 7752.11342029 7752.11342029 7752.11342029 7752.11342029
   7752.11342029 7752.11342029 7752.11342029 7752.11342029 7752.11342029
      0.            0.        ]
  [   0.            0.            0.         7752.11342029 7752.11342029
   7752.11342029 7752.11342029 7752.11342029 7752.113

0.9973913331081093

[[[   0.         7752.11341541 7752.11334925 7752.11304518    0.
      0.            0.            0.            0.            0.
      0.            0.        ]
  [7752.11341166 7752.11328069 7752.11077078 7752.08992478 7752.11161051
      0.            0.            0.         7750.79994561 7752.09378192
      0.            0.        ]
  [7752.11327572 7752.11015468 7752.0318668  7750.81435262 7752.05073736
   7752.11155506 7752.10923256 7751.59631267 7714.95757622 7751.62782536
   7752.1047643  7752.11330569]
  [7752.11206125 7752.07015516 7750.53759094 7697.98433836 7750.76562835
   7752.08554842 7752.11300071 7752.09449309 7750.95694305 7752.09572072
   7752.11295665 7752.11341341]
  [7752.11327816 7752.11028519 7752.03298125 7750.84760091 7752.05179105
   7752.11167417 7752.11339766 7752.11276741 7752.07272097 7752.11281902
      0.            0.        ]
  [   0.            0.            0.         7752.08646052 7752.1113896
   7752.11335052 7752.11341941 775

0.9977487123727106

--
0.5
--
0.5
--
0.125
--
[[[   0.         7752.1134019  7752.11318567 7752.11234098    0.
      0.            0.            0.            0.            0.
      0.            0.        ]
  [7752.11338787 7752.11296213 7752.10593557 7752.05734458 7752.10823848
      0.            0.            0.         7749.59278335 7752.06597909
      0.            0.        ]
  [7752.11294559 7752.10421006 7751.92078253 7749.62720469 7751.96451596
   7752.10808864 7752.10326032 7751.11544228 7698.64947875 7751.17834139
   7752.0926297  7752.11308864]
  [7752.10957107 7752.01109357 7749.11456674 7673.84709198 7749.53778772
   7752.04694344 7752.11221175 7752.06815358 7749.92108356 7752.07111215
   7752.11208822 7752.11339712]
  [7752.11294974 7752.10451314 7751.92231609 7749.68869744 7751.96690813
   7752.10844075 7752.11334458 7752.11155816 7752.01762547 7752.11170296
      0.            0.        ]
  [   0.            0.            0.         7752.04864628 7752.10761649
   7752

...

0.9976169668571511

--
2.5
--
2.5
--
0.5
--
[[[   0.         7752.08363424 7751.99298315 7751.89754122    0.
      0.            0.            0.            0.            0.
      0.            0.        ]
  [7752.06424403 7751.90238443 7751.01183645 7749.23620636 7751.18372965
      0.            0.            0.         7716.75264454 7749.26196852
      0.            0.        ]
  [7751.89126881 7750.78860257 7743.6780675  7718.24306031 7745.09808128
   7751.18375358 7751.44000535 7736.89250099 7572.23701534 7738.23675941
   7750.95450588 7752.0419331 ]
  [7751.50146388 7747.46671516 7713.82635547 7477.65123883 7717.70689619
   7748.76178443 7751.8720755  7749.74241415 7725.59551951 7749.9145193
   7751.86028898 7752.09688747]
  [7751.86439762 7750.6902714  7743.08740472 7718.63088949 7745.13561338
   7751.2749635  7752.06554109 7751.79537265 7748.13709857 7751.81336327
      0.            0.        ]
  [   0.            0.            0.         7748.48854432 7751.10133989
   7751.98

0.9985143688617104

[[[   0.         7751.95823901 7751.63599011 7751.39336588    0.
      0.            0.            0.            0.            0.
      0.            0.        ]
  [7751.86457546 7751.31726787 7748.95358978 7745.44120523 7749.21268189
      0.            0.            0.         7692.49450034 7745.05427493
      0.            0.        ]
  [7751.26597961 7748.34839835 7734.07237396 7695.63545264 7736.63850311
   7749.25297869 7750.33432448 7725.27123932 7531.73485693 7727.90959037
   7749.32849573 7751.86814258]
  [7750.25514433 7741.88197285 7689.81621004 7412.93692756 7695.22581841
   7744.43051473 7751.31519382 7746.58618619 7710.08280253 7747.0053657
   7751.30004607 7752.03794985]
  [7751.11973519 7747.89096813 7732.1872159  7695.81164414 7736.64512181
   7749.57167493 7751.90642228 7751.13511239 7743.81469719 7751.18058833
      0.            0.        ]
  [   0.            0.            0.         7743.4018516  7748.99549907
   7751.57595822 7752.07473129 775

0.9987597906323841

--
4.0
--
4.0
--
0.5
--
[[[   0.         7751.83547965 7751.33668044 7751.00314789    0.
      0.            0.            0.            0.            0.
      0.            0.        ]
  [7751.67315903 7750.84492886 7747.54836216 7743.12434958 7747.77715408
      0.            0.            0.         7680.66482806 7742.34948167
      0.            0.        ]
  [7750.7560623  7746.69411976 7728.6620589  7684.72887615 7731.73913288
   7747.86275412 7749.57587431 7719.28725225 7516.08759266 7722.63682161
   7748.30501884 7751.73085446]
  [7749.35309572 7738.6146613  7678.4724124  7388.30271707 7684.46557717
   7741.8175527  7750.88812422 7744.65501168 7702.97315706 7745.23049067
   7750.88136399 7751.983311  ]
  [7750.49486773 7745.93066257 7725.87696418 7684.66149287 7731.70628462
   7748.35888142 7751.76461254 7750.65695177 7741.43104732 7750.71844624
      0.            0.        ]
  [   0.            0.            0.         7740.20807041 7747.47683695
   7751.2

0.999055991982429

[[[   0.         7751.42206597 7750.45522636 7749.9165908     0.
      0.            0.            0.            0.            0.
      0.            0.        ]
  [7751.04257551 7749.50548267 7744.08917585 7737.85339608 7744.03082277
      0.            0.            0.         7658.20837962 7735.9168032
      0.            0.        ]
  [7749.29395366 7742.64704051 7717.27411826 7664.21922414 7721.16368633
   7744.27401066 7747.64755481 7707.33372592 7490.62137486 7712.15641459
   7745.90898328 7751.34076238]
  [7746.98782834 7731.44327436 7657.43533811 7349.35866105 7664.33791341
   7735.9404314  7749.71095755 7740.25757585 7690.09517672 7741.19856167
   7749.7585228  7751.8056454 ]
  [7748.6602625  7740.99788933 7712.28036211 7663.40785708 7721.00593188
   7745.25914201 7751.32169053 7749.40950803 7736.49107817 7749.50323053
      0.            0.        ]
  [   0.            0.            0.         7732.76933405 7743.54951552
   7750.20031198 7751.92257811 7751

0.9991518752527792



In [5]:
from tabulate import tabulate
print_production_matrix = np.zeros((len(t_report),7))

for j in range(0,len(t_report)):
    print_production_matrix[j,0] = t_report[j]
    print_production_matrix[j,1] = p_28[j]
    print_production_matrix[j,2] = pwf_28[j]
    print_production_matrix[j,3] = Jw_28[j]
    print_production_matrix[j,4] = p_33[j]
    print_production_matrix[j,5] = q_33[j]
    print_production_matrix[j,6] = Jw_33[j]
    

print(tabulate(print_production_matrix, headers=['Time', 'P(2,8)','Pwf(2,8)','Jw(2,8)','P(3,3)','q(3,3)','Jw(3,3)']))

  Time    P(2,8)    Pwf(2,8)    Jw(2,8)    P(3,3)       q(3,3)    Jw(3,3)
------  --------  ----------  ---------  --------  -----------  ---------
 0       7752.11     7752.11     inf      7752.11  0               inf
 0.125   7732.72     6982.99    4001.46   7724.01  7.24604e+06    4203.03
 0.25    7714.96     6965.6     4003.41   7697.98  7.14179e+06    4206.04
 0.375   7698.65     6949.63    4005.21   7673.85  7.04497e+06    4208.85
 0.5     7683.65     6934.93    4006.87   7651.42  6.95489e+06    4211.47
 0.75    7657.9      6909.72    4009.73   7612.29  6.7975e+06     4216.05
 1       7635.66     6887.94    4012.21   7577.97  6.65918e+06    4220.1
 1.25    7616.3      6868.98    4014.37   7547.71  6.53701e+06    4223.67
 1.5     7599.32     6852.36    4016.27   7520.9   6.42862e+06    4226.86
 2       7572.24     6825.84    4019.31   7477.65  6.25344e+06    4232.01
 2.5     7550.14     6804.21    4021.8    7442.27  6.10982e+06    4236.25
 3       7531.73     6786.19    4023.88   

In [6]:
from tabulate import tabulate
print_report_matrix = np.zeros((len(t_report),3))

for j in range(0,len(t_report)):
    print_report_matrix[j,0] = t_report[j]
    print_report_matrix[j,1] = p_res_average[j]
    print_report_matrix[j,2] = MB_error[j]
    

print(tabulate(print_report_matrix, headers=['Time', 'Average P','MB error']))

  Time    Average P    MB error
------  -----------  ----------
 0          7752.11    0
 0.125      7751.33    0.996964
 0.25       7750.56    0.997394
 0.375      7749.8     0.997749
 0.5        7749.04    0.998042
 0.75       7747.57    0.99695
 1          7746.12    0.997576
 1.25       7744.69    0.998034
 1.5        7743.27    0.998374
 2          7740.5     0.997617
 2.5        7737.77    0.998162
 3          7735.07    0.998517
 3.5        7732.41    0.99876
 4          7729.77    0.998931
 4.5        7727.14    0.999057
 5          7724.54    0.999152


In [9]:
print(dt_report)
print(p)

[0, 0.125, 0.125, 0.125, 0.125, 0.25, 0.25, 0.25, 0.25, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
[[[   0.         7751.11461998 7749.86355554 7749.21411814    0.
      0.            0.            0.            0.            0.
      0.            0.        ]
  [7750.58241868 7748.63527767 7742.08383212 7734.96779753 7741.75044167
      0.            0.            0.         7647.66798331 7732.26226611
      0.            0.        ]
  [7748.33446325 7740.31295537 7711.47585112 7654.6786922  7715.65497972
   7742.10994453 7746.4818037  7701.44565292 7479.97581747 7707.00660903
   7744.56266691 7751.08394391]
  [7745.53232861 7727.6323083  7647.74270169 7333.70666641 7655.0070856
   7732.75597863 7748.95694114 7737.84973244 7684.26888103 7738.99499258
   7749.05724661 7751.67605047]
  [7747.43553699 7738.08740958 7705.21850908 7653.38043426 7715.4150274
   7743.40506895 7751.00953396 7748.64749505 7734.00163708 7748.75594679
      0.            0.        ]
  [   0.            0.            0. 

In [8]:
import numpy as np
import matplotlib.pyplot as plt

cor = []

x_cor = 0
for ix in range (0,Nx):
    if ix == 0:
        x_cor += dx[0,ix]/2    
    else:
        x_cor += dx[0,ix-1]/2 + dx[0,ix]/2
    y_cor = 0
    for iy in range (0,Ny):
        if iy == 0:
            y_cor += dy[iy,0]/2  
            cor.append([x_cor,y_cor])
        else:
            y_cor += dy[iy,0]/2 + dy[iy-1,0]/2
            cor.append([x_cor,y_cor])
                
    
cor_array = np.array(cor)

pressure = []

for ix in range (0,Nx):
    for iy in range (0,Ny):
        pressure.append(p_new[iy,ix])

pressure_array = np.array(pressure)

grid_x, grid_y = np.mgrid[0:x_cor:100j, 0:y_cor:100j]

from scipy.interpolate import griddata
grid_z0 = griddata(cor_array, pressure_array, (grid_x, grid_y), method='nearest')

plt.imshow(grid_z0.T, extent=(0,x_cor,y_cor,0),vmin=6500, vmax=7750,cmap="PuBu")
plt.title('well block pressure')
plt.colorbar()
plt.show()

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
A = [[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16]]
A_del = np.delete(A,[1,3],0)
A_del = np.delete(A_del,[1,3],1)
print(A_del)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

cor = []

x_cor = 0
for ix in range (0,Nx):
    if ix == 0:
        x_cor += dx[0,ix]/2    
    else:
        x_cor += dx[0,ix-1]/2 + dx[0,ix]/2
    y_cor = 0
    for iy in range (0,Ny):
        if iy == 0:
            y_cor += dy[iy,0]/2  
            cor.append([x_cor,y_cor])
        else:
            y_cor += dy[iy,0]/2 + dy[iy-1,0]/2
            cor.append([x_cor,y_cor])
                
    
cor_array = np.array(cor)

porosity = []

for ix in range (0,Nx):
    for iy in range (0,Ny):
        porosity.append(phi[iy,ix])

porosity_array = np.array(porosity)

grid_x, grid_y = np.mgrid[0:x_cor:100j, 0:y_cor:100j]

from scipy.interpolate import griddata
grid_z0 = griddata(cor_array, porosity_array, (grid_x, grid_y), method='nearest')

plt.imshow(grid_z0.T, extent=(0,x_cor,y_cor,0),vmin=0.18, vmax=0.24,cmap="PuBu_r")
plt.title('porosity')
plt.colorbar()
plt.show()

In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

x_array = []
y_array = []
z_array = []

x_cor = 0
for ix in range (0,Nx):
    if ix == 0:
        x_cor += dx[0,ix]/2 
        x_array.append(x_cor)
    else:
        x_cor += dx[0,ix-1]/2 + dx[0,ix]/2
        x_array.append(x_cor)

y_cor = 0
for iy in range (0,Ny):
    if iy == 0:
        y_cor += dy[iy,0]/2  
        y_array.append(y_cor)
    else:
        y_cor += dy[iy,0]/2 + dy[iy-1,0]/2
        y_array.append(y_cor)


for ix in range (0,Nx):
    for iy in range (0,Ny):
        z_array.append(middle[iy,ix])


X, Y = np.meshgrid(x_array, y_array)
Z = -middle
pressure = p_new
Z[abs(Z)<1] = np.nan
    
fig = plt.figure()

ax = Axes3D(fig)
#ax = fig.add_subplot(111, projection='3d')
ax.view_init(45,45)
ax.set_xlim3d(abs(np.max(X)),0)
ax.set_ylim3d(abs(np.max(Y)),0)
ax.set_zlim3d(-9450,-9250)
# here we create the surface plot, but pass V through a colormap
# to create a different color for each patch
norm = matplotlib.colors.Normalize(vmin=6500, vmax=7750)
ax.plot_surface(X, Y, Z, facecolors = cm.viridis(norm(pressure)),linewidth=0.1)
#fig.colorbar(fig,shrink=0.5, aspect=5)
plt.title('Pressure Distribution 3D',fontsize='14', y=1.1)
plt.xlabel('x (ft)')
plt.ylabel('y (ft)')

pressure_range = np.arange(6500,7750)
m = cm.ScalarMappable(cmap=cm.viridis)
m.set_array(pressure_range)
plt.colorbar(m)

In [ ]:
import numpy as np

z = np.arange(20).reshape(5, 4)
mask = np.zeros(z.shape, dtype=bool)

mask[3, 2] = True

print(z)
print(np.ma.masked_array(z, mask))

In [ ]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

x_array = []
y_array = []
z_array = []

x_cor = 0
for ix in range (0,Nx):
    if ix == 0:
        x_cor += dx[0,ix]/2 
        x_array.append(x_cor)
    else:
        x_cor += dx[0,ix-1]/2 + dx[0,ix]/2
        x_array.append(x_cor)

y_cor = 0
for iy in range (0,Ny):
    if iy == 0:
        y_cor += dy[iy,0]/2  
        y_array.append(y_cor)
    else:
        y_cor += dy[iy,0]/2 + dy[iy-1,0]/2
        y_array.append(y_cor)


for ix in range (0,Nx):
    for iy in range (0,Ny):
        z_array.append(middle[iy,ix])


X, Y = np.meshgrid(x_array, y_array)
Z = -middle
porosity = phi
Z[abs(Z)<1] = np.nan
    
fig = plt.figure()

ax = Axes3D(fig)
#ax = fig.add_subplot(111, projection='3d')
ax.view_init(45,45)
ax.set_xlim3d(abs(np.max(X)),0)
ax.set_ylim3d(abs(np.max(Y)),0)
ax.set_zlim3d(-9450,-9250)
# here we create the surface plot, but pass V through a colormap
# to create a different color for each patch
norm = matplotlib.colors.Normalize(vmin=0.18, vmax=0.25)
ax.plot_surface(X, Y, Z, facecolors = cm.PuBu_r(norm(porosity)),linewidth=0.1)
#fig.colorbar(fig,shrink=0.5, aspect=5)
plt.title('Porosity Distribution 3D',fontsize='14', y=1.1)
plt.xlabel('x (ft)')
plt.ylabel('y (ft)')

porosity_range = np.arange(0.18,0.25,0.01)
m = cm.ScalarMappable(cmap=cm.PuBu_r)
m.set_array(porosity_range)
plt.colorbar(m)

In [ ]:
R_matrix = np.zeros((Ny,Nx))
for iy in range (0,Ny):
    for ix in range (0,Nx):
        R_matrix[iy,ix] = R(iy,ix,'0')
        
R_matrixW = np.zeros((Ny,Nx))
for iy in range (0,Ny):
    for ix in range (0,Nx):
        R_matrixW[iy,ix] = R(iy,ix,'W')
        
print(R_matrix)
print(R_matrixW)
print(null)

In [ ]:
dRdp_matrixC = np.zeros((Ny,Nx))
for iy in range (0,Ny):
    for ix in range (0,Nx):
        dRdp_matrixC[iy,ix] = dR_dp(iy,ix,'C')
print(dRdp_matrixC)

dRdp_matrixW = np.zeros((Ny,Nx))
for iy in range (0,Ny):
    for ix in range (0,Nx):
        dRdp_matrixW[iy,ix] = dR_dp(iy,ix,'W')
print(dRdp_matrixW)

In [ ]:
print(residual_dp)